In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
import math
import pycatch22
import pickle

In [2]:
data1=pd.read_csv('./data/EURUSD_M1.csv',sep='\t')
tmp=data1.columns+'_EURUSD'
tmp=['Time']+tmp[1:].tolist()
data1.columns=tmp

data2=pd.read_csv('./data/AUDUSD_M1.csv',sep='\t')
tmp=data2.columns+'_AUDUSD'
tmp=['Time']+tmp[1:].tolist()
data2.columns=tmp

data3=pd.read_csv('./data/EURJPY_M1.csv',sep='\t')
tmp=data3.columns+'_EURJPY'
tmp=['Time']+tmp[1:].tolist()
data3.columns=tmp

data4=pd.read_csv('./data/GBPAUD_M1.csv',sep='\t')
tmp=data4.columns+'_GBPAUD'
tmp=['Time']+tmp[1:].tolist()
data4.columns=tmp

data5=pd.read_csv('./data/USDCAD_M1.csv',sep='\t').drop_duplicates('Time')
tmp=data5.columns+'_USDCAD'
tmp=['Time']+tmp[1:].tolist()
data5.columns=tmp

In [3]:
data=data1.merge(data2,how='left',on='Time').merge(data3,how='left',on='Time').merge(data4,how='left',on='Time').merge(data5,how='left',on='Time')
data=data.ffill()
data.loc[:,'Time']=pd.to_datetime(data.Time)
data.loc[:,'date_day']=data.Time.dt.to_period('D')
data.loc[:,'ID']=np.arange(data.shape[0])
for day in data.date_day.unique():
    data.loc[data.date_day==day,'ID_day']=np.arange(sum(data.date_day==day))

In [9]:
def preprocessing(data,N_x=60,N_x2=10,N_y=30):
     
    colnames=pycatch22.catch22_all(data.iloc[:,2],catch24=True)['names']
    
    data_new=pd.DataFrame()
    
    for col in data.columns[1:-3]:
        colnames_new=[(x+'_n1_'+col) for x in colnames]
        colnames_new2=[(x+'_n2_'+col) for x in colnames]
        for n in range(0,(100000-N_x-N_y)):
            if (n%10000==0):
                print(n)
            data_new.loc[n,colnames_new]=(pycatch22.catch22_all(data.loc[data.ID.between(n,n+N_x-1),col],catch24=True))['values']
            data_new.loc[n,colnames_new2]=(pycatch22.catch22_all(data.loc[data.ID.between(n+(N_x-N_x2),n+N_x-1),col],catch24=True))['values']
            
    for n in range(0,(100000-N_x-N_y)):
        if (n%10000==0):
            print(n)

        data.loc[n,'Time_pred']=data.loc[data.ID==(n+N_x+1),'Time'].iloc[0]
        data.loc[n,'pred_open']=data.loc[data.ID==(n+N_x+1),'Open_EURUSD'].iloc[0]
        data.loc[n,'pred_max']=data.loc[data.ID.between(n+N_x+1,n+N_x+N_y-1),'High_EURUSD'].max()
        data.loc[n,'pred_min']=data.loc[data.ID.between(n+N_x+1,n+N_x+N_y-1),'Low_EURUSD'].min()
        data.loc[n,'same_day']=data.loc[data.ID.between(n,n+N_x+N_y-1),'date_day'].nunique()==1
    data.loc[:,'gain']=(data.pred_max-data.pred_open)
    return data,data_new

In [4]:
def preprocessing(data,N_x=60,N_x2=10,N_y=30):
     
    colnames=pycatch22.catch22_all(data.iloc[:,2],catch24=True)['names']
    
    data_new=pd.DataFrame()
    
    for col in data.columns[1:-3]:
        colnames_new=[(x+'_n1_'+col) for x in colnames]
        colnames_new2=[(x+'_n2_'+col) for x in colnames]
        for n in range(N_x,(100000-N_y)):
            if (n%10000==0):
                print(n)
            data_new.loc[n,colnames_new]=(pycatch22.catch22_all(data.loc[data.ID.between(n-N_x,n-1),col],catch24=True))['values']
            data_new.loc[n,colnames_new2]=(pycatch22.catch22_all(data.loc[data.ID.between(n-N_x2,n-1),col],catch24=True))['values']
            
    for n in range(N_x,(100000-N_y)):
        if (n%10000==0):
            print(n)

        data.loc[n,'pred_open']=data.loc[data.ID==n,'Open_EURUSD'].iloc[0]
        data.loc[n,'pred_max']=data.loc[data.ID.between(n+1,n+N_y-1),'High_EURUSD'].max()
        data.loc[n,'pred_min']=data.loc[data.ID.between(n+1,n+N_y-1),'Low_EURUSD'].min()
        data.loc[n,'same_day']=data.loc[data.ID.between(n-N_x,n+N_y-1),'date_day'].nunique()==1
    data.loc[:,'gain']=(data.pred_max-data.pred_open)
    return data,data_new

In [5]:
data,data_new=preprocessing(data,N_x=60,N_x2=10,N_y=10)
with open('./data/features_60_10_10.pickle', 'wb') as f:
    pickle.dump([data,data_new], f)

10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
5000

In [6]:
data=data1.merge(data2,how='left',on='Time').merge(data3,how='left',on='Time').merge(data4,how='left',on='Time').merge(data5,how='left',on='Time')
data=data.ffill()
data.loc[:,'Time']=pd.to_datetime(data.Time)
data.loc[:,'date_day']=data.Time.dt.to_period('D')
data.loc[:,'ID']=np.arange(data.shape[0])
for day in data.date_day.unique():
    data.loc[data.date_day==day,'ID_day']=np.arange(sum(data.date_day==day))

data,data_new=preprocessing(data,N_x=60,N_x2=10,N_y=20)
with open('./data/features_60_10_20.pickle', 'wb') as f:
    pickle.dump([data,data_new], f)

10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
5000

In [7]:
data=data1.merge(data2,how='left',on='Time').merge(data3,how='left',on='Time').merge(data4,how='left',on='Time').merge(data5,how='left',on='Time')
data=data.ffill()
data.loc[:,'Time']=pd.to_datetime(data.Time)
data.loc[:,'date_day']=data.Time.dt.to_period('D')
data.loc[:,'ID']=np.arange(data.shape[0])
for day in data.date_day.unique():
    data.loc[data.date_day==day,'ID_day']=np.arange(sum(data.date_day==day))

data,data_new=preprocessing(data,N_x=90,N_x2=10,N_y=30)
with open('./data/features_90_10_30.pickle', 'wb') as f:
    pickle.dump([data,data_new], f)

10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
50000
60000
70000
80000
90000
10000
20000
30000
40000
5000